Simplest simplest, let's start with a linear regression...

In [ ]:
from sklearn.model_selection import KFold, cross_val_score 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def train_and_evaluate_model(features_df):
    print("\n--- Training and Evaluating Linear Regression Model with 10-Fold Cross-Validation ---")
    MIN_SAMPLES_FOR_CV = 10 
    if features_df.empty or len(features_df) < MIN_SAMPLES_FOR_CV:
        print(f"Not enough data for 10-fold CV. Need {MIN_SAMPLES_FOR_CV}, found {len(features_df)}.")
        return

    y = features_df['delta_OD']
    X = features_df.drop(columns=['delta_OD', 'OD'])

    if X.empty: print("No features available."); return

    X_filled = X.fillna(X.mean())
    if X_filled.isna().any().any():
        cols_to_drop = X_filled.columns[X_filled.isna().all()].tolist()
        if cols_to_drop:
            X_filled = X_filled.drop(columns=cols_to_drop)
            print(f"Dropped all-NaN columns: {cols_to_drop}")
        if X_filled.empty or X_filled.isna().any().any() :
             print("Features are still problematic after NaN handling."); return
    
    y_aligned = y.loc[X_filled.index]

    pipeline = Pipeline([('scaler', StandardScaler()), ('regressor', LinearRegression())])
    cv = KFold(n_splits=10, shuffle=True, random_state=42)

    mse_scores = cross_val_score(pipeline, X_filled, y_aligned, scoring='neg_mean_squared_error', cv=cv)
    mae_scores = cross_val_score(pipeline, X_filled, y_aligned, scoring='neg_mean_absolute_error', cv=cv)
    r2_scores = cross_val_score(pipeline, X_filled, y_aligned, scoring='r2', cv=cv)

    print("\nCross-Validation Model Evaluation Metrics (10-fold):")
    print(f"  Mean Squared Error (MSE): {-np.mean(mse_scores):.4f} (+/- {np.std(mse_scores):.4f})")
    print(f"  Root Mean Squared Error (RMSE): {np.sqrt(-np.mean(mse_scores)):.4f}")
    print(f"  Mean Absolute Error (MAE): {-np.mean(mae_scores):.4f} (+/- {np.std(mae_scores):.4f})")
    print(f"  R-squared (R²): {np.mean(r2_scores):.4f} (+/- {np.std(r2_scores):.4f})")

    # Get cross-validated predictions for plotting
    delta_od_cv_predictions = cross_val_predict(pipeline, X_filled, y_aligned, cv=cv)

    # --- Parity Plot ---
    plt.figure(figsize=(8, 8))
    plt.scatter(y_aligned, delta_od_cv_predictions, alpha=0.7, edgecolors='k')
    min_val = min(y_aligned.min(), delta_od_cv_predictions.min())
    max_val = max(y_aligned.max(), delta_od_cv_predictions.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2) # y=x line
    plt.xlabel("Actual delta_OD")
    plt.ylabel("Predicted delta_OD (Cross-Validated)")
    plt.title("Parity Plot: Actual vs. Predicted delta_OD")
    plt.grid(True)
    plt.show()

    # --- OD Time Course Plot ---
    # The first actual OD value *before* any deltas in features_df
    # OD_current = OD_previous + delta_OD  => OD_previous = OD_current - delta_OD
    initial_actual_od_for_course = features_df['OD'].iloc[0] - features_df['delta_OD'].iloc[0]

    # Actual OD course
    actual_od_course = np.concatenate(([initial_actual_od_for_course], features_df['OD'].values))
    
    # Predicted OD course (reconstructed)
    predicted_od_course = [initial_actual_od_for_course]
    current_predicted_od = initial_actual_od_for_course
    for pred_delta in delta_od_cv_predictions:
        current_predicted_od += pred_delta
        predicted_od_course.append(current_predicted_od)
    predicted_od_course = np.array(predicted_od_course)

    # Timestamps for plotting
    # Need one timestamp before the first one in features_df.index
    plot_timestamps = features_df.index
    if len(plot_timestamps) > 1:
        time_diff = plot_timestamps[1] - plot_timestamps[0]
        initial_timestamp = plot_timestamps[0] - time_diff
    else: # Fallback if only one delta_OD point (so one timestamp in features_df)
        initial_timestamp = plot_timestamps[0] - pd.Timedelta(hours=1) # Arbitrary fallback

    full_plot_timestamps = pd.to_datetime([initial_timestamp] + plot_timestamps.tolist())


    plt.figure(figsize=(12, 6))
    plt.plot(full_plot_timestamps, actual_od_course, label='Actual OD', marker='o', linestyle='-')
    plt.plot(full_plot_timestamps, predicted_od_course, label='Predicted OD (Reconstructed from CV deltas)', marker='x', linestyle='--')
    plt.xlabel("Time")
    plt.ylabel("Optical Density (OD)")
    plt.title("Actual vs. Reconstructed Predicted OD Time Course")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


    # --- Feature Importance (from model trained on all data) ---
    print("\n--- Training Final Model on All Data for Feature Importance ---")
    pipeline.fit(X_filled, y_aligned)
    coefficients_scaled = pipeline.named_steps['regressor'].coef_
    feature_names = X_filled.columns
    coefficients_df = pd.DataFrame(coefficients_scaled, feature_names, columns=['Coefficient'])
    sorted_coefficients = coefficients_df.reindex(coefficients_df['Coefficient'].abs().sort_values(ascending=False).index)
    print("\nFeature Importance (Coefficients from model trained on all data, features were scaled):")
    print(sorted_coefficients)


In [ ]:
def train_and_evaluate_model(features_df):
    print("\n--- Training and Evaluating Linear Regression Model with 10-Fold Cross-Validation ---")
    # For 10-fold CV, we need at least 10 samples.
    MIN_SAMPLES_FOR_CV = 10 
    if features_df.empty or len(features_df) < MIN_SAMPLES_FOR_CV:
        print(f"Not enough data for 10-fold cross-validation. Need at least {MIN_SAMPLES_FOR_CV} samples, found {len(features_df)}.")
        return

    y = features_df['delta_OD']
    X = features_df.drop(columns=['delta_OD', 'OD'])

    if X.empty:
        print("No features available. Cannot train model.")
        return

    X_filled = X.fillna(X.mean())
    if X_filled.isna().any().any():
        print("Warning: NaN values still present in features after mean imputation. Dropping these columns.")
        cols_to_drop = X_filled.columns[X_filled.isna().all()].tolist()
        X_filled = X_filled.drop(columns=cols_to_drop)
        if X_filled.empty:
            print("No features left after dropping all-NaN columns. Cannot train model.")
            return
        print(f"Dropped all-NaN columns: {cols_to_drop}")
    
    # Align y with X_filled in case any rows were dropped (though fillna shouldn't drop rows)
    # However, if X_filled becomes empty or significantly changes shape, y needs to align.
    # This alignment is crucial if X_filled.dropna() were used. With fillna, index should remain.
    y_aligned = y.loc[X_filled.index]


    # Create a pipeline that first scales the data then applies linear regression
    pipeline = Pipeline([
        # ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ])

    # Define the K-fold cross-validator
    cv = KFold(n_splits=10, shuffle=True, random_state=42)

    # Perform cross-validation for different metrics
    # Note: cross_val_score returns negative MSE and MAE, so we'll flip the sign
    mse_scores = cross_val_score(pipeline, X_filled, y_aligned, scoring='neg_mean_squared_error', cv=cv)
    mae_scores = cross_val_score(pipeline, X_filled, y_aligned, scoring='neg_mean_absolute_error', cv=cv)
    r2_scores = cross_val_score(pipeline, X_filled, y_aligned, scoring='r2', cv=cv)

    # Calculate and print mean and std dev of the metrics
    mean_mse = -np.mean(mse_scores)
    std_mse = np.std(mse_scores)
    mean_rmse = np.sqrt(mean_mse) # RMSE from mean MSE
    mean_mae = -np.mean(mae_scores)
    std_mae = np.std(mae_scores)
    mean_r2 = np.mean(r2_scores)
    std_r2 = np.std(r2_scores)

    print("\nCross-Validation Model Evaluation Metrics (10-fold):")
    print(f"  Mean Squared Error (MSE): {mean_mse:.4f} (+/- {std_mse:.4f})")
    print(f"  Root Mean Squared Error (RMSE): {mean_rmse:.4f} (derived from mean MSE)")
    print(f"  Mean Absolute Error (MAE): {mean_mae:.4f} (+/- {std_mae:.4f})")
    print(f"  R-squared (R²): {mean_r2:.4f} (+/- {std_r2:.4f})")

    # For feature importance, train the final model on all available data
    print("\n--- Training Final Model on All Data for Feature Importance ---")
    pipeline.fit(X_filled, y_aligned)
    
    # Extract coefficients from the regressor step in the pipeline
    # The scaler step in the pipeline already scaled the features
    coefficients_scaled = pipeline.named_steps['regressor'].coef_
    
    feature_names = X_filled.columns
    coefficients_df = pd.DataFrame(coefficients_scaled, feature_names, columns=['Coefficient'])
    
    # Sort by absolute magnitude to see strongest impacts
    sorted_coefficients = coefficients_df.reindex(coefficients_df['Coefficient'].abs().sort_values(ascending=False).index)
    
    print("\nFeature Importance (Coefficients from model trained on all data, features were scaled):")
    print(sorted_coefficients)
    print("\nNote: Coefficients represent the change in delta_OD for a one-unit change in the (scaled) feature.")


In [10]:
summary_statistics_df = pd.read_csv('cyano_culture_datasets/04-15-2025 culture/combined_dataset_summary_statistics.csv')
train_and_evaluate_model(summary_statistics_df)


--- Training and Evaluating Linear Regression Model with 10-Fold Cross-Validation ---

Cross-Validation Model Evaluation Metrics (10-fold):
  Mean Squared Error (MSE): 0.0018 (+/- 0.0055)
  Root Mean Squared Error (RMSE): 0.0429 (derived from mean MSE)
  Mean Absolute Error (MAE): 0.0044 (+/- 0.0093)
  R-squared (R²): -400.7283 (+/- 1199.9058)

--- Training Final Model on All Data for Feature Importance ---

Feature Importance (Coefficients from model trained on all data, features were scaled):
                                           Coefficient
std_pH                                   -7.987909e-01
min_pH                                   -4.222010e-01
mean_pH                                   3.965542e-01
first_last_diff_pH                       -7.274191e-02
mean_temperature (C)                      5.049678e-02
median_pH                                 4.581971e-02
median_temperature (C)                   -2.533162e-02
max_pH                                   -2.039574e-02
min_

# Shall we throw a 1D-CNN at it?